In [50]:
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import json
import os
import re
from scipy.optimize import curve_fit
from scipy.signal import argrelextrema
from scipy.interpolate import interp1d
from textwrap import wrap
import math
import imageio
debug = True

def read_file(filename):
    
    scale = int(filename.split("/")[-1].split("-")[0])
    tile_count = int(filename.split("/")[-1].split("-")[1])
    with open(filename, 'r') as f:
        l = [[num for num in line.split(' ')][:-1] for line in f]

        lambda_vector = np.array(l[0]).astype('float64')
        q_vector = np.reciprocal(lambda_vector)
        tau_vector = np.array(l[1]).astype('float64')
        
        non_ave_ISF = []
        for i in range(2,len(l)):
            non_ave_ISF_at_t = np.reshape((np.array(l[i]).astype('float64')),(int(scale/2+1),scale))
            # non_ave_ISF_at_t = non_ave_ISF_at_t/np.amax(non_ave_ISF_at_t)                             
            non_ave_ISF.append(non_ave_ISF_at_t) 
            
    
    if debug:
        print(len(q_vector))
        print(len(tau_vector))
        print(len(non_ave_ISF))
        print(len(non_ave_ISF))

    return (lambda_vector, q_vector, tau_vector, non_ave_ISF, scale, tile_count)

In [40]:
filename = "/cicutagroup/yz655/cuda_run_result/testing/test_nonAve_ISF_compressed_fps10_WinterWheat1_2022-07-25-140826-0000/1024-0-nonAve"
# print(read_file(filename))
lambda_vector, q_vector, tau_vector, non_ave_ISF, scale, tile_count = read_file(filename)

16
100
100
100


In [75]:
def plot_Iqtau_heatmap(tau_number, tau_vector, scale, non_ave_ISF, filename, plot_path):
# def plot_Iqtau_heatmap(tau_number, tau_vector, scale, non_ave_ISF, filename):
    
    fig,ax = plt.subplots(1,1)

    # im = ax.imshow(non_ave_ISF[tau_number], cmap='hot', interpolation='nearest', aspect='auto', extent=[0,scale, 0, (scale/2+1)])
    im = ax.imshow(non_ave_ISF[tau_number], cmap='cool', interpolation='nearest', extent=[0,scale, 0, (scale/2+1)],  norm="log")
    # ax.set_yscale('log')

    # Where we want the ticks, in pixel locations
    xticks = range(0,scale)
    yticks = range(0,scale)
    # What those pixel locations correspond to in data coordinates.
    # Also set the float format here
    xticklabels = ["{:6.2f}".format(i) for i in xticks]
    yticklabels = ["{:6.2f}".format(i) for i in yticks]

    plt.xlabel('pixel')
    plt.ylabel('pixel')
    plt.title('ISF map at t='+str((tau_number+1)/10)+'s')
    fig.colorbar(im)
    
    # plt.show()
    
    if not os.path.exists(plot_path):
        os.makedirs(plot_path)

    plt.savefig(plot_path+'heatmap'+'_tau='+str((tau_number+1)/10)+'.jpg', bbox_inches='tight')
    plt.close()
    

In [76]:
plot_path = "/cicutagroup/yz655/cuda_run_result/testing/test_nonAve_ISF_compressed_fps10_WinterWheat1_2022-07-25-140826-0000/heatmap_plots/"
for tau_number in range(100):
    plot_Iqtau_heatmap(tau_number, tau_vector, scale, non_ave_ISF, filename, plot_path)

In [81]:


def plot_gif(png_dir):
    images = []
    for file_name in sorted(os.listdir(png_dir)):
        if file_name.endswith('.jpg'):
            file_path = os.path.join(png_dir, file_name)
            images.append(imageio.imread(file_path))

    # Make it pause at the end so that the viewers can ponder
    for _ in range(10):
        images.append(imageio.imread(file_path))

    imageio.mimsave(png_dir+'movie.gif', images)

In [51]:
matplotlib.scale.get_scale_names()

['asinh', 'function', 'functionlog', 'linear', 'log', 'logit', 'symlog']

In [80]:
def make_heatmap_gif(filename, plot_path):
    
    # print(read_file(filename))
    lambda_vector, q_vector, tau_vector, non_ave_ISF, scale, tile_count = read_file(filename)
    
    for tau_number in range(len(tau_vector)):
        plot_Iqtau_heatmap(tau_number, tau_vector, scale, non_ave_ISF, filename, plot_path)
        
    plot_gif(plot_path)
    

In [82]:
filename = "/cicutagroup/yz655/cuda_run_result/testing/test_nonAve_ISF_DJI_0050/1024-0-nonAve"
plot_path = "/cicutagroup/yz655/cuda_run_result/testing/test_nonAve_ISF_DJI_0050/heatmap_plots/"

make_heatmap_gif(filename, plot_path)


16
100
100
100


In [83]:
plot_gif(plot_path)

/tmp/ipykernel_6087/2861615859.py:6: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(file_path))
/tmp/ipykernel_6087/2861615859.py:10: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(file_path))


In [86]:
filename = "/cicutagroup/yz655/cuda_run_result/testing/test_nonAve_ISF_DJI_0052/1024-0-nonAve"
plot_path = "/cicutagroup/yz655/cuda_run_result/testing/test_nonAve_ISF_DJI_0052/heatmap_plots/"

make_heatmap_gif(filename, plot_path)

16
100
100
100


In [87]:
plot_gif(plot_path)

/tmp/ipykernel_6087/2861615859.py:6: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(file_path))
/tmp/ipykernel_6087/2861615859.py:10: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(file_path))
